# 2. Intro to Extended Kalman Filter Project

## 2.1 - Project Introduction

Now that you have learned how the extended Kalman filter works, you are going to implement the extended Kalman filter in C++. We are providing `simulated lidar and radar measurements detecting a bicycle that travels around your vehicle`. You will use a Kalman filter, lidar measurements and radar measurements to track the bicycle's position and velocity.

The first step is to `download the simulator`, which contains all the projects for Self-Driving Car Nanodegree. More detailed instruction about setting up the simulator with `uWebSocketIO` can be found at the end of this section.

`Lidar` measurements are `red circles`,` radar` measurements are `blue circles` with an arrow pointing in the direction of the observed angle, and `estimation markers` are `green triangles`. 

The video below shows what the simulator looks like when a c++ script is using its Kalman filter to track the object. The simulator provides the script the measured data (either lidar or radar), and the script feeds back the measured estimation marker, and RMSE values from its Kalman filter.

https://youtu.be/bb2IPCmJGwM

## 2.2 - Example of Tracking with Lidar

In [4]:
HTML("""
<video width="640" height="360" controls>
  <source src='_Videos/3_Data_collected.mp4'>
</video>
""")

# 3. Data File for EKF project

## 3.1 - Explanation of the Data File

The github repo contains one data file:

> obj_pose-laser-radar-synthetic-input.txt

Here is a screenshot of the first data file:

![](_Images/3_1_screenshot.png)
Screenshot of Data File

The simulator will be using this data file, and feed main.cpp values from it one line at a time.

Each row represents a sensor measurement where the first column tells you if the measurement comes from radar (R) or lidar (L).

For a row containing radar data, the columns are: sensor_type, rho_measured, phi_measured, rhodot_measured, timestamp, x_groundtruth, y_groundtruth, vx_groundtruth, vy_groundtruth, yaw_groundtruth, yawrate_groundtruth.

For a row containing lidar data, the columns are: sensor_type, x_measured, y_measured, timestamp, x_groundtruth, y_groundtruth, vx_groundtruth, vy_groundtruth, yaw_groundtruth, yawrate_groundtruth.

Whereas radar has three measurements (rho, phi, rhodot), lidar has two measurements (x, y).

You will use the measurement values and timestamp in your Kalman filter algorithm. Groundtruth, which represents the actual path the bicycle took, is for calculating root mean squared error.

You do not need to worry about `yaw` and `yaw rate` ground truth values.

# 3.2 - Reading in the Data

We have provided code that will read in and parse the data files for you. This code is in the main.cpp file. The main.cpp file creates instances of a MeasurementPackage.

If you look inside 'main.cpp', you will see code like:

```C++
MeasurementPackage meas_package;
meas_package.sensor_type_ = MeasurementPackage::LASER;
meas_package.raw_measurements_ = VectorXd(2);
meas_package.raw_measurements_ << px, py;
meas_package.timestamp_ = timestamp;
```

and

```C++
vector<VectorXd> ground_truth;
VectorXd gt_values(4);
gt_values(0) = x_gt;
gt_values(1) = y_gt; 
gt_values(2) = vx_gt;
gt_values(3) = vy_gt;
ground_truth.push_back(gt_values);
```

The code reads in the data file line by line. The measurement data for each line gets pushed onto a measurement_pack_list. The ground truth $[p_x, p_y, v_x, v_y]$ for each line in the data file gets pushed ontoground_truthso RMSE can be calculated later from tools.cpp.

# 4. File Structure

# 4.1 - Overview of a Kalman Filter: Initialize, Predict, Update

To review what we learned in the extended Kalman filter lectures, let's discuss the three main steps for programming a Kalman filter:

- `initializing` Kalman filter variables
- `predicting` where our object is going to be after a time step $\Delta{t}$
- `updating` where our object is based on sensor measurements

Then the prediction and update steps repeat themselves in a loop.

To measure how well our Kalman filter performs, we will then calculate root mean squared error comparing the Kalman filter results with the provided ground truth.

These `three steps (initialize, predict, update)` plus calculating `RMSE` encapsulate the entire extended Kalman filter project.

# 4.2 - Files in the Github src Folder

The files you need to work with are in the src folder of the github repository.

- `main.cpp` - communicates with the `Term 2 Simulator` receiving data measurements, calls a function to run the Kalman filter, calls a function to calculate RMSE
- `FusionEKF.cpp` - initializes the filter, calls the predict function, calls the update function
- `kalman_filter.cpp`- defines the `predict function`, the `update function for lidar`, and t`he update function for radar`
- `tools.cpp`v- function to calculate RMSE and the Jacobian matrix

The only files you need to modify are `FusionEKF.cpp`, `kalman_filter.cpp`, and `tools.cpp`.

# 4.3 - How the Files Relate to Each Other

Here is a brief overview of what happens when you run the code files:

- `Main.cpp` reads in the data and sends a `sensor measurement` to `FusionEKF.cpp`
- `FusionEKF.cpp` takes the sensor data and initializes variables and updates variables. The Kalman filter equations are not in this file. FusionEKF.cpp has a variable called ekf_, which is an instance of a KalmanFilter class. The ekf_ will hold the matrix and vector values. You will also use the ekf_ instance to call the predict and update equations.
- `The KalmanFilter` class is defined in kalman_filter.cpp and kalman_filter.h. You will only need to modify 'kalman_filter.cpp', which contains functions for the prediction and update steps.

# 5. Main.cpp

Here we will discuss the main.cpp file. Although you will not need to modify this file, the project is easier to implement once you understand what the file is doing. As a suggestion, open the github repository for the project and look at the code files simultaneously with this lecture slide.

**Main.cpp**

You do not need to modify the main.cpp, but let's discuss what the file does.

The **Term 2 simulator** is a client, and the c++ program software is a web server.

We already discussed how main.cpp reads in the sensor data. Recall that main.cpp reads in the sensor data line by line from the client and stores the data into a measurement object that it passes to the Kalman filter for processing. Also a ground truth list and an estimation list are used for tracking RMSE.

main.cpp is made up of several functions within main(), these all handle the uWebsocketIO communication between the simulator and it's self.

Here is the main protocol that main.cpp uses for uWebSocketIO in communicating with the simulator.

```C++
INPUT: values provided by the simulator to the c++ program

["sensor_measurement"] => the measurement that the simulator observed (either lidar or radar)


OUTPUT: values provided by the c++ program to the simulator

["estimate_x"] <= kalman filter estimated position x
["estimate_y"] <= kalman filter estimated position y
["rmse_x"]
["rmse_y"]
["rmse_vx"]
["rmse_vy"]
```

All the main code loops in h.onMessage(), to have access to intial variables that we created at the beginning of main(), we pass pointers as arguments into the header of h.onMessage().

For example:

```C++
h.onMessage([&fusionEKF,&tools,&estimations,&ground_truth]
            (uWS::WebSocket<uWS::SERVER> ws, char *data, size_t length, 
             uWS::OpCode opCode)
```

The rest of the arguments in h.onMessage are used to set up the server.

```C++
 // Create a Fusion EKF instance
  FusionEKF fusionEKF;

  // used to compute the RMSE later
  vector<VectorXd> estimations;
  vector<VectorXd> ground_truth;

  //Call the EKF-based fusion
  fusionEKF.ProcessMeasurement(meas_package); 
```

The code is:

- creating an instance of the FusionEKF class
- Receiving the measurement data calling the ProcessMeasurement() function. ProcessMeasurement() is responsible for the initialization of the Kalman filter as well as calling the prediction and update steps of the Kalman filter. You will be implementing the ProcessMeasurement() function in FusionEKF.cpp:

Finally,

The rest of main.cpp will output the following results to the simulator:

- estimation position
- calculated RMSE

main.cpp will call a function to calculate root mean squared error:

```C++
  // compute the accuracy (RMSE)
  Tools tools;
  cout << "Accuracy - RMSE:" << endl << tools.CalculateRMSE(estimations, ground_truth) << endl;
```

You will implement an RMSE function in the tools.cpp file.

# 6. Project Code

Compiling and Running Your Code

Take a look at the github repo [README file](https://github.com/udacity/CarND-Extended-Kalman-Filter-Project/blob/master/README.md) for instructions on how to compile and run your code.

# 7. Tips and Tricks

## 7.1 - Summary of What Needs to Be Done

1. In tools.cpp, fill in the functions that calculate root mean squared error (RMSE) and the Jacobian matrix.
2. Fill in the code in FusionEKF.cpp. You'll need to initialize the Kalman Filter, prepare the Q and F matrices for the prediction step, and call the radar and lidar update functions.
3. In kalman_filter.cpp, fill out the Predict(), Update(), and UpdateEKF() functions.

## 7.2 - Tips and Tricks

**Review the Previous Lessons**

- Review the previous lessons! Andrei, Dominik and co. have given you everything you need. In fact, you've built most of an Extended Kalman Filter already! Take a look at the programming assignments and apply the techniques you used to this project.

**No Need to Tune Parameters**

- The R matrix values and Q noise values are provided for you. There is no need to tune these parameters for this project.

**Initializing the State Vector**

- You'll need to initialize the state vector with the first sensor measurement.
- Although radar gives velocity data in the form of the range rate $\dot{\rho}$, a radar measurement does not contain enough information to determine the state variable velocities $v_x$ and $v_y$. You can, however, use the radar measurements $\rho$ and $\phi$ to initialize the state variable locations $p_x$ and $p_y$.

**Calculating y = z - H * x'**

For lidar measurements, the error equation is y = z - H * x'. For radar measurements, the functions that map the x vector [px, py, vx, vy] to polar coordinates are non-linear. Instead of using H to calculate y = z - H * x', for radar measurements you'll have to use the equations that map from cartesian to polar coordinates: y = z - h(x').

**Normalizing Angles**

In C++, atan2() returns values between -pi and pi. When calculating phi in y = z - h(x) for radar measurements, the resulting angle phi in the y vector should be adjusted so that it is between -pi and pi. The Kalman filter is expecting small angle values between the range -pi and pi. HINT: when working in radians, you can add $2\pi$ or subtract $2\pi$ until the angle is within the desired range.

**Avoid Divide by Zero throughout the Implementation**

Before and while calculating the Jacobian matrix Hj, make sure your code avoids dividing by zero. For example, both the x and y values might be zero or px*px + py*py might be close to zero. What should be done in those cases?

**Test Your Implementation**

Test! We're giving you the ability to analyze your output data and calculate RMSE. As you make changes, keep testing your algorithm! If you are getting stuck, add print statements to pinpoint any issues. But please remove extra print statements before turning in the code.

## 7.3 - Ideas for Standing out!

The Kalman Filter general processing flow that you've learned in the preceding lessons gives you the basic knowledge needed to track an object. However, there are ways that you can make your algorithm `more efficient`!

- Dealing with the first frame, in particular, offers opportunities for improvement.
- Experiment and see how low your RMSE can go!
- Try removing radar or lidar data from the filter. Observe how your estimations change when running against a `single sensor type!` Do the results make sense given what you know about the nature of radar and lidar data?
- We give you starter code, but you are not required to use it! You may want to start from scratch if: You want a bigger challenge! You want to `redesign the project architecture`. There are many valid design patterns for approaching the Kalman Filter algorithm. Feel free to experiment and try your own! You want to use a different coding style, eg. functional programming. While C++ code naturally tends towards being object-oriented in nature, it's perfectly reasonable to attempt a functional approach. Give it a shot and maybe you can improve its efficiency!

# ** 8. Project Resources

# 9. Project Instruction for workspace

## 9.1 - Accessing and using the workspace:

- Navigate to the workspace node.
- Navigate to the repository `CarND-Extended-Kalman-Filter-Project` using the file and directory menu on the left.
- Run `./install-ubuntu.sh` in the terminal window to make sure the necessary dependencies are installed (you may need to run chmod u+x install-ubuntu.sh to make the file executable).
- Complete the TODO in `FusionEKF.cpp`, `FusionEKF.h`, `kalman_filter.cpp`, `kalman_filter.h`, `tools.cpp` and `tools.h` using the text editor in the workspace.
- Navigate to the project repository in the terminal.

The main program can be built and run by doing the following from the project top directory:

1. mkdir build && cd build
2. cmake .. && make
3. ./ExtendedKF

Click on the "Simulator" button in the bottom of the Udacity workspace, which will open a new virtual desktop. You should see a "Simulator" link on the virtual desktop. `Double-click the "Simulator"` link in that desktop to start the simulator.

Important: You need to `open a terminal` before attempting to run the simulator.

## 9.2 - Extended Kalman Filter Simulator

- In the main menu `screen` select Project `1/2: EKF and UKF`.
- Once the scene is loaded you can hit the `START button` to observe how the object moves and how measurement markers are positioned in the data set. Also for more experimentation, "Data set 2" is included which is a `reversed version` of "Data set 1", also the second data set starts with a radar measurement where the first data set starts with a lidar measurement. - At any time you can press the PAUSE button, to pause the scene or hit the RESTART button to reset the scene. Also, the `ARROW KEYS` can be used to `move the camera` around, and the top left `ZOOM IN/OUT` buttons can be used to focus the camera. Pressing the `ESCAPE KEY` returns to the simulator main menu

**NOTES:**

- Currently hitting Restart or switching between data sets only refreshes the simulator state and not the Kalman Filter's saved results. The current procedure for refreshing the Kalman Filter is to `close the connection, ctrl+c and reopen it, ./ExtendedKF`. If you don't do this when trying to run a different data set or running the same data set multiple times in a row, `the RMSE values will become large` because of the previous different filter results still being observed in memory.
- The `two functions in tools.cpp` need to be implemented for the simulator to avoid throwing a `segmentation fault` - make sure to get those built out in order to test in the simulator!

# ** 10. Project Instructions for local setup

# ** 12. uWebSocketIO Starter guide

## ** 15. Environment Setup (Mac)

# ** 16. Compiling and Running the Project